In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")


from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

# LN4 이어서
## 2 결합확률과 주변확률 부터
- 필기 참고
## 3 파이썬: 함수
- 내장된 함수는 그냥 쓰면 되고
- 내장되지 않은 건 내가 만들어 쓰면 됨

In [2]:
min(0,1) # 정의된 함수

0

In [3]:
max(0,1)

1

In [5]:
df=pd.DataFrame([[4,9]]*3,columns=['A','B'])
df

,A,B
0,4,9
1,4,9
2,4,9


In [6]:
# 이 데이터 프레임 전체에 뭔가 연산을 해주고 싶다 -> apply 쓰면 됨, apply 도 내장함수 built-in function
df.apply(np.sqrt)

,A,B
0,2.0,3.0
1,2.0,3.0
2,2.0,3.0


#### 사용자 정의 함수
- def으로 시작, 이름 정의
- indent된 상태에서 함수 정의
- x<0이 아니면은 if에 맞춰서 쓰면됨

In [7]:
def negative(x):
    if x<0:
        return 'negative'
    return 'non-negative' # x<0이 아닐때

In [8]:
negative(-3)

'negative'

In [9]:
negative(5)

'non-negative'

In [10]:
def f1(x, a=1, b=1): # 값을 3개 입력하겠다. 혹시 a와 b를 입력하지 않으면 자동으로 각각 1을 사용해라. 라는 뜻
    return a*b*x

In [11]:
f1(2,3,4)

24

In [12]:
f1(2)

2

In [13]:
f1(2,4)

8

## 4 공공데이터 포털에서 증권거래소 종목 가져오기

- 주가, 회사의 정보가 필요할 것이다 .
- 증권거래소에서 이 정보를 가져오자


In [15]:
df=pd.read_excel('c:/temp/stock_list.xlsx')
df

,표준코드,단축코드,한글 종목명,한글 종목약명,영문 종목명,상장일,시장구분,증권구분,소속부,주식종류,액면가,상장주식수
0,KR7098120009,098120,(주)마이크로컨텍솔루션,마이크로컨텍솔,"Micro Contact Solution Co.,Ltd.",2008/09/23,KOSDAQ,주권,벤처기업부,보통주,500,8312766
1,KR7131100000,131100,(주)스카이이앤엠,스카이이앤엠,"SKY E&M Co., Ltd.",2011/06/14,KOSDAQ,주권,관리종목(소속부없음),보통주,500,11642629
2,KR7009520008,009520,(주)포스코엠텍,포스코엠텍,"POSCO M-TECH CO.,LTD.",1997/11/10,KOSDAQ,주권,우량기업부,보통주,500,41642703
3,KR7095570008,095570,AJ네트웍스보통주,AJ네트웍스,"AJ Networks Co.,Ltd.",2015/08/21,KOSPI,주권,NaN,보통주,1000,46822295
4,KR7006840003,006840,AK홀딩스보통주,AK홀딩스,"AK Holdings, Inc.",1999/08/11,KOSPI,주권,NaN,보통주,5000,13247561
...,...,...,...,...,...,...,...,...,...,...,...,...
2603,KR7000542001,000547,흥국화재해상보험2우선주(신형),흥국화재2우B,HeungkukFire&MarineIns(2PB),1999/08/09,KOSPI,주권,NaN,신형우선주,5000,153600
2604,KR7000540005,000540,흥국화재해상보험보통주,흥국화재,HeungkukFire&MarineInsurance,1974/12/05,KOSPI,주권,NaN,보통주,5000,64242645
2605,KR7003280005,003280,흥아해운보통주,흥아해운,HeungaShipping,1976/06/29,KOSPI,주권,NaN,보통주,500,240424899
2606,KR7037440005,037440,희림종합건축사사무소,희림,Heerim Architects & Planners,2000/02/03,KOSDAQ,주권,중견기업부,보통주,500,13922475


In [18]:
# 보고싶은 칼럼만 보려면...
df1=df[['단축코드','한글 종목명']] # 반드시 column이름과 똑같이 쳐야한다, 칼럼 두 개 이상 가져올 때는 [[]] double square braket? 암튼 써야함
df1

,단축코드,한글 종목명
0,098120,(주)마이크로컨텍솔루션
1,131100,(주)스카이이앤엠
2,009520,(주)포스코엠텍
3,095570,AJ네트웍스보통주
4,006840,AK홀딩스보통주
...,...,...
2603,000547,흥국화재해상보험2우선주(신형)
2604,000540,흥국화재해상보험보통주
2605,003280,흥아해운보통주
2606,037440,희림종합건축사사무소


In [19]:
df1.head()

,단축코드,한글 종목명
0,098120,(주)마이크로컨텍솔루션
1,131100,(주)스카이이앤엠
2,009520,(주)포스코엠텍
3,095570,AJ네트웍스보통주
4,006840,AK홀딩스보통주


In [20]:
df1.tail()

,단축코드,한글 종목명
2603,000547,흥국화재해상보험2우선주(신형)
2604,000540,흥국화재해상보험보통주
2605,003280,흥아해운보통주
2606,037440,희림종합건축사사무소
2607,238490,힘스


In [25]:
# Q: 삼성전자 보통주의 단축코드는 뭔가여
# 어디에 있는지 그 숫자(iloc)를 모르고 레이블 사용해야하니까 loc 써야함

#isin: 한글종목명 안에 삼성전자보통주가 있으면 그 row를 가져와
df1.loc[df1['한글 종목명'].isin(['삼성전자보통주'])]

,단축코드,한글 종목명
836,005930,삼성전자보통주


### krx 대신 open api 로 종목 데이터 가져오기

In [26]:
# api를 이용해서 or krx 든 상장종목 정보 불러올 수 있다.

#Servicekey="ㅇㅇㅇ"
#numOfRows=3000
#url="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD="+str(gu_code)+"&DEAL_YMD="+str(month)+"&numOfRows="+str(numOfRows)+"&serviceKey="+Servicekey

# 이렇게 받으면 krx로 받은거랑 똑같음
# 강의 노트 보고 꼭 해보기!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [27]:
# Q: 시장구분이라는 변수를 이용해서 3000개 회사 중 kospi와 kosdaq에 상장된 회사 숫자와 그 비율은? 
# A: frequency table을 만들어보자

#value_counts: 범주형 변수의 frequency table을 만들어주는 함수
pd.value_counts(df['시장구분'])

KOSDAQ    1533
KOSPI      943
KONEX      132
Name: 시장구분, dtype: int64

In [28]:
# 비율은?  -> normalize=True 사용
pd.value_counts(df['시장구분'],normalize=True)

KOSDAQ    0.587807
KOSPI     0.361580
KONEX     0.050613
Name: 시장구분, dtype: float64

In [29]:
# frequency table 다른 방법으로 : pd의 crosstab

pd.crosstab(df['시장구분'])

TypeError: crosstab() missing 1 required positional argument: 'columns'

In [30]:
# 에러나는 이유: crosstab은 원래 변수를 두개 입력해야함. two way table 만드는 함수임.
# 임의로 하나 써버리자 
pd.crosstab(df['시장구분'], 'no')

col_0,no
시장구분,
KONEX,132
KOSDAQ,1533
KOSPI,943


In [31]:
# 상장종목정보 url 올려주시면 혼자 꼭 해보기